In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import pandas as pd
import requests
import json
import numpy as np
from datetime import datetime
from google.colab import files

# 1.0 Load Cleaned 2019 Ridership Data

In [3]:
data_dir = '/content/drive/My Drive/MIE498 Thesis/0_Data'

In [4]:
data_2019_all_cleaned = pd.read_csv("{}/cleaned_data_2019.csv".format(data_dir), header=0)
print(data_2019_all_cleaned.shape)
data_2019_all_cleaned.head()

(2439047, 22)


,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Bike Id,Start Year,Start Month,Start Hour,Start Day of Week,Start Holiday,End Year,End Month,End Hour,End Day of Week,End Holiday,Start Day of Year,Start Week of Year
0,4581278,1547.0,7021,2019-01-01 00:08:00,Bay St / Albert St,7233,2019-01-01 00:33:00,King / Cowan Ave - SMART,Annual Member,1296,2019,1,0,1,1,2019,1,0,1,1,1,1
1,4581279,1112.0,7160,2019-01-01 00:10:00,King St W / Tecumseth St,7051,2019-01-01 00:29:00,Wellesley St E / Yonge St (Green P),Annual Member,2947,2019,1,0,1,1,2019,1,0,1,1,1,1
2,4581280,589.0,7055,2019-01-01 00:15:00,Jarvis St / Carlton St,7013,2019-01-01 00:25:00,Scott St / The Esplanade,Annual Member,2293,2019,1,0,1,1,2019,1,0,1,1,1,1
3,4581281,259.0,7012,2019-01-01 00:16:00,Elizabeth St / Edward St (Bus Terminal),7235,2019-01-01 00:20:00,Bay St / College St (West Side) - SMART,Annual Member,283,2019,1,0,1,1,2019,1,0,1,1,1,1
4,4581282,281.0,7041,2019-01-01 00:19:00,Edward St / Yonge St,7257,2019-01-01 00:24:00,Dundas St W / St. Patrick St,Annual Member,1799,2019,1,0,1,1,2019,1,0,1,1,1,1


# 2.0 Retrieve Bike Station Information

In [9]:
# Station Information: 
url = "https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information"
params = {"name": "station_information"}
response = requests.get(url, params = params)
# print(package["result"])
print(response.status_code)
print(response.json())

200
{'last_updated': 1632968040, 'ttl': 0, 'data': {'stations': [{'station_id': '7000', 'name': 'Fort York  Blvd / Capreol Ct', 'physical_configuration': 'REGULAR', 'lat': 43.639832, 'lon': -79.395954, 'altitude': 0.0, 'address': 'Fort York  Blvd / Capreol Ct', 'capacity': 35, 'rental_methods': ['KEY', 'TRANSITCARD', 'CREDITCARD', 'PHONE'], 'groups': [], 'obcn': '647-643-9607', 'nearby_distance': 500.0}, {'station_id': '7001', 'name': 'Wellesley Station Green P', 'physical_configuration': 'REGULAR', 'lat': 43.66496415990742, 'lon': -79.38355031526893, 'altitude': 0.0, 'address': 'Yonge / Wellesley', 'post_code': 'M4Y 1G7', 'capacity': 17, 'rental_methods': ['KEY', 'TRANSITCARD', 'CREDITCARD', 'PHONE'], 'groups': [], 'obcn': '416-617-9576', 'nearby_distance': 500.0}, {'station_id': '7002', 'name': 'St. George St / Bloor St W', 'physical_configuration': 'REGULAR', 'lat': 43.667333, 'lon': -79.399429, 'altitude': 0.0, 'address': 'St. George St / Bloor St W', 'capacity': 19, 'rental_method

In [7]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [8]:
# Station Status: 
# url = "https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_status"
# params = {"name": "station_status"}
# response = requests.get(url, params = params)
# # print(package["result"])
# print(response.status_code)
# print(response.json())
# jprint(response.json())

200
{'last_updated': 1632968036, 'ttl': 1, 'data': {'stations': [{'station_id': '7000', 'num_bikes_available': 25, 'num_bikes_available_types': {'mechanical': 25, 'ebike': 0}, 'num_bikes_disabled': 0, 'num_docks_available': 9, 'num_docks_disabled': 0, 'last_reported': 1632967809, 'is_charging_station': False, 'status': 'IN_SERVICE', 'is_installed': 1, 'is_renting': 1, 'is_returning': 1}, {'station_id': '7001', 'num_bikes_available': 4, 'num_bikes_available_types': {'mechanical': 4, 'ebike': 0}, 'num_bikes_disabled': 2, 'num_docks_available': 11, 'num_docks_disabled': 0, 'last_reported': 1632967837, 'is_charging_station': False, 'status': 'IN_SERVICE', 'is_installed': 1, 'is_renting': 1, 'is_returning': 1}, {'station_id': '7002', 'num_bikes_available': 3, 'num_bikes_available_types': {'mechanical': 3, 'ebike': 0}, 'num_bikes_disabled': 0, 'num_docks_available': 16, 'num_docks_disabled': 0, 'last_reported': 1632967878, 'is_charging_station': False, 'status': 'IN_SERVICE', 'is_installed':

In [10]:
ts = int(json.loads(response.content)['last_updated'])
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2021-09-30 02:14:00


In [11]:
bikeshare_stations = pd.DataFrame(json.loads(response.content)['data']['stations'])[['station_id', 'name', 'lat', 'lon', 'capacity', 'address']].astype({'station_id': 'float64',})
bikeshare_stations['station_id'] = bikeshare_stations['station_id'].astype("int64")
print(bikeshare_stations.shape)
bikeshare_stations.head(15)

(610, 6)


,station_id,name,lat,lon,capacity,address
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,35,Fort York Blvd / Capreol Ct
1,7001,Wellesley Station Green P,43.664964,-79.383550,17,Yonge / Wellesley
2,7002,St. George St / Bloor St W,43.667333,-79.399429,19,St. George St / Bloor St W
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761,15,Madison Ave / Bloor St W
4,7004,University Ave / Elm St,43.656518,-79.389099,11,University Ave / Elm St
5,7005,King St W / York St,43.648001,-79.383177,19,King St W / York St
6,7006,Bay St / College St (East Side),43.660439,-79.385525,11,Bay St / College St (East Side)
7,7007,College St / Huron St,43.658148,-79.398167,18,College St / Huron St
8,7008,Wellesley St / Queen's Park Cres,43.663376,-79.392125,19,Wellesley St W / Queen's Park Cres
9,7009,King St E / Jarvis St,43.650325,-79.372287,25,King St E / Jarvis St


In [12]:
jprint(response.json()['data']['stations'][0:3])

[
    {
        "address": "Fort York  Blvd / Capreol Ct",
        "altitude": 0.0,
        "capacity": 35,
        "groups": [],
        "lat": 43.639832,
        "lon": -79.395954,
        "name": "Fort York  Blvd / Capreol Ct",
        "nearby_distance": 500.0,
        "obcn": "647-643-9607",
        "physical_configuration": "REGULAR",
        "rental_methods": [
            "KEY",
            "TRANSITCARD",
            "CREDITCARD",
            "PHONE"
        ],
        "station_id": "7000"
    },
    {
        "address": "Yonge / Wellesley",
        "altitude": 0.0,
        "capacity": 17,
        "groups": [],
        "lat": 43.66496415990742,
        "lon": -79.38355031526893,
        "name": "Wellesley Station Green P",
        "nearby_distance": 500.0,
        "obcn": "416-617-9576",
        "physical_configuration": "REGULAR",
        "post_code": "M4Y 1G7",
        "rental_methods": [
            "KEY",
            "TRANSITCARD",
            "CREDITCARD",
            "PHON

In [25]:
bikeshare_stations.to_csv('bike_stations_info_20200930.csv', index=False, header=True)
files.download('bike_stations_info_20200930.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 3.0 Check 1: Station ID and Station Names

In [67]:
bikeshare_stations = pd.read_csv("{}/bike_stations_info_20200930.csv".format(data_dir), header=0)
print(bikeshare_stations.shape)  # 2021-09-30 02:14:00
bikeshare_stations.head()

(610, 6)


,station_id,name,lat,lon,capacity,address
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,35,Fort York Blvd / Capreol Ct
1,7001,Wellesley Station Green P,43.664964,-79.383550,17,Yonge / Wellesley
2,7002,St. George St / Bloor St W,43.667333,-79.399429,19,St. George St / Bloor St W
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761,15,Madison Ave / Bloor St W
4,7004,University Ave / Elm St,43.656518,-79.389099,11,University Ave / Elm St


In [68]:
data_2019_all_cleaned.columns

Index(['Trip Id', 'Trip Duration', 'Start Station Id', 'Start Time',
       'Start Station Name', 'End Station Id', 'End Time', 'End Station Name',
       'User Type', 'Bike Id', 'Start Year', 'Start Month', 'Start Hour',
       'Start Day of Week', 'Start Holiday', 'End Year', 'End Month',
       'End Hour', 'End Day of Week', 'End Holiday', 'Start Day of Year',
       'Start Week of Year'],
      dtype='object')

In [69]:
bikeshare_stations.columns

Index(['station_id', 'name', 'lat', 'lon', 'capacity', 'address'], dtype='object')

In [70]:
# Get unique station IDs from dataframes
unique_start_ids_2019 = data_2019_all_cleaned['Start Station Id'].unique().reshape(-1, 1)
unique_end_ids_2019 = data_2019_all_cleaned['End Station Id'].unique().reshape(-1, 1)
unique_ids_reference = bikeshare_stations['station_id'].unique().reshape(-1, 1)
print('Number of Unique START Station IDs (2019 Bike Ridership Data): ', len(unique_start_ids_2019))
print('Number of Unique END Station IDs (2019 Bike Ridership Data): ', len(unique_end_ids_2019))
print('Number of Station IDs (Reference from Station Information): ', len(unique_ids_reference))

Number of Unique START Station IDs (2019 Bike Ridership Data):  469
Number of Unique END Station IDs (2019 Bike Ridership Data):  468
Number of Station IDs (Reference from Station Information):  610


In [71]:
# Only need to check the start station IDs as all end station IDs are covered by the start station IDs 
print(np.setdiff1d(unique_start_ids_2019, unique_end_ids_2019))
print(np.setdiff1d(unique_end_ids_2019, unique_start_ids_2019))

[7393]
[]


In [72]:
# Check difference between the start station IDs from 2019 ridership data and the station information
diff_inRef_notIn2019 = np.setdiff1d(unique_ids_reference, unique_start_ids_2019)
print('diff_inRef_notIn2019: ', len(diff_inRef_notIn2019), '\n', diff_inRef_notIn2019)
diff_in2019_notInRef = np.setdiff1d(unique_start_ids_2019, unique_ids_reference)
print('diff_in2019_notInRef: ', len(diff_in2019_notInRef), '\n', diff_in2019_notInRef)

diff_inRef_notIn2019:  154 
 [7514 7515 7516 7517 7518 7519 7520 7521 7522 7523 7524 7525 7526 7527
 7528 7529 7530 7531 7533 7534 7535 7536 7537 7538 7539 7540 7541 7542
 7543 7544 7545 7546 7547 7548 7549 7551 7552 7553 7554 7555 7556 7557
 7558 7559 7560 7561 7562 7563 7564 7565 7566 7567 7568 7569 7570 7571
 7572 7575 7576 7577 7578 7579 7580 7581 7582 7583 7584 7585 7586 7587
 7588 7589 7590 7591 7593 7594 7595 7596 7597 7598 7599 7600 7601 7602
 7603 7604 7605 7606 7607 7608 7609 7610 7611 7612 7613 7614 7615 7616
 7617 7618 7619 7620 7621 7622 7623 7624 7625 7626 7627 7629 7630 7631
 7632 7633 7634 7635 7636 7637 7639 7640 7641 7642 7643 7644 7645 7646
 7647 7648 7650 7651 7653 7654 7655 7656 7657 7658 7659 7660 7662 7663
 7664 7665 7666 7667 7668 7670 7671 7675 7676 7677 7678 7679 7680 7681]
diff_in2019_notInRef:  13 
 [7116 7218 7219 7254 7322 7393 7394 7402 7456 7470 7481 7510 7511]


In [73]:
154-13+469

610

In [74]:
start_station_2019_dict = pd.Series(data_2019_all_cleaned['Start Station Name'].values,index=data_2019_all_cleaned['Start Station Id']).to_dict()
reference_station_dict = pd.Series(bikeshare_stations['name'].values,index=bikeshare_stations['station_id']).to_dict()

In [75]:
# start_station_2019_dict

In [76]:
def check_missing_ref_id(start_station_2019_dict, reference_station_dict):
  missing_ref_id = 0
  for key, val in start_station_2019_dict.items():
    if reference_station_dict.get(key) is None:
      missing_ref_id += 1
      print("{} Station ID not in Reference: {}, Station Name: {}".format(missing_ref_id, key, val))

In [77]:
check_missing_ref_id(start_station_2019_dict, reference_station_dict)

1 Station ID not in Reference: 7322, Station Name: King St W / Brant St
2 Station ID not in Reference: 7254, Station Name: Borden St / Bloor St W - SMART
3 Station ID not in Reference: 7116, Station Name: Parliament St / Bloor St E
4 Station ID not in Reference: 7393, Station Name: 55 Magnificent Rd. Garage
5 Station ID not in Reference: 7219, Station Name: Wolfpack - 7219
6 Station ID not in Reference: 7402, Station Name: Wellington St W / Bathurst St
7 Station ID not in Reference: 7470, Station Name: York St / Lake Shore Blvd W
8 Station ID not in Reference: 7456, Station Name: Belmont St / Davenport Rd - SMART
9 Station ID not in Reference: 7481, Station Name: Westmount Ave / St Clair Ave W - SMART
10 Station ID not in Reference: 7218, Station Name: Make Invisible - 7218
11 Station ID not in Reference: 7510, Station Name: Princes Gate / Nunavut Dr
12 Station ID not in Reference: 7394, Station Name: Toronto Bike Shop
13 Station ID not in Reference: 7511, Station Name: PBSC-OPS


|#|Station ID |lat|lon|
|---|---|---|---|
|1|7322|43.645133|-79.397639|
|2|7254|43.665810|-79.408639|
|3|7116|43.672196|-79.370728|
|4|7393|43.612978|-79.511731|
|5|7219||
|6|7402|43.642769|-79.402129|
|7|7470|43.641782|-79.380877|
|8|7456|43.674865|-79.392841|
|9|7481|43.678374|-79.441843|
|10|7218||
|11|7510|43.633499|-79.414126|
|12|7394||
|13|7511||

### Modification 1: Add some missing stations information

In [78]:
correct_station_ids_list = [7322, 7254, 7116, 7393, 7402, 7470, 7456, 7481, 7510]
correct_lat_list = [43.645133, 43.665810, 43.672196, 43.612978, 43.642769, 43.641782, 43.674865, 43.678374, 43.633499]
correct_lon_list = [-79.397639, -79.408639, -79.370728, -79.511731, -79.402129, -79.380877, -79.392841, -79.441843, -79.414126]
correct_names = ['King St W / Brant St', 'Borden St / Bloor St W - SMART', 'Parliament St / Bloor St E', '55 Magnificent Rd. Garage', 'Wellington St W / Bathurst St',
                 'York St / Lake Shore Blvd W', 'Belmont St / Davenport Rd - SMART', 'Westmount Ave / St Clair Ave W - SMART', 'Princes Gate / Nunavut Dr']

In [79]:
modified_bikeshare_stations = bikeshare_stations.copy()

for i, id in enumerate(correct_station_ids_list):
  series_obj = pd.Series([id, correct_names[i], correct_lat_list[i], correct_lon_list[i], np.nan, None], index=modified_bikeshare_stations.columns)
  modified_bikeshare_stations = modified_bikeshare_stations.append(series_obj, ignore_index=True)

modified_bikeshare_stations.tail(10)

,station_id,name,lat,lon,capacity,address
609,7681,25 Booth Ave,43.654484,-79.341057,12.0,25 BOOTH AVE
610,7322,King St W / Brant St,43.645133,-79.397639,NaN,None
611,7254,Borden St / Bloor St W - SMART,43.665810,-79.408639,NaN,None
612,7116,Parliament St / Bloor St E,43.672196,-79.370728,NaN,None
613,7393,55 Magnificent Rd. Garage,43.612978,-79.511731,NaN,None
614,7402,Wellington St W / Bathurst St,43.642769,-79.402129,NaN,None
615,7470,York St / Lake Shore Blvd W,43.641782,-79.380877,NaN,None
616,7456,Belmont St / Davenport Rd - SMART,43.674865,-79.392841,NaN,None
617,7481,Westmount Ave / St Clair Ave W - SMART,43.678374,-79.441843,NaN,None
618,7510,Princes Gate / Nunavut Dr,43.633499,-79.414126,NaN,None


# 4.0 Check 2: Mismatched Station Names with Same Station ID

In [80]:
def check_mismatch_names(start_station_2019_dict, reference_station_dict):
  mismatch_names = 0
  for key, val in start_station_2019_dict.items():
    if reference_station_dict.get(key) is not None:
      if val != reference_station_dict.get(key):
        mismatch_names += 1
        print("{} Mismatch Station Names: ID={}, Data={}, Reference={}".format(mismatch_names, key, val, reference_station_dict.get(key)))

In [81]:
check_mismatch_names(start_station_2019_dict, reference_station_dict)

1 Mismatch Station Names: ID=7381, Data=Roehampton Ave / Yonge St, Reference=Yonge St / Orchard View Blvd
2 Mismatch Station Names: ID=7324, Data=King St W / Charlotte St (West), Reference=King St W / Charlotte St (West Side)
3 Mismatch Station Names: ID=7051, Data=Wellesley St E / Yonge St (Green P), Reference=Mutual St / Shuter St
4 Mismatch Station Names: ID=7342, Data=Morse St / Eastern Ave - SMART, Reference=Morse St / Eastern Ave
5 Mismatch Station Names: ID=7127, Data=Bay St / Davenport Rd, Reference=Bay St / Scollard St
6 Mismatch Station Names: ID=7372, Data=King St W / Portland St, Reference=Adelaide St W / Portland St
7 Mismatch Station Names: ID=7001, Data=Lower Jarvis St / The Esplanade, Reference=Wellesley Station Green P
8 Mismatch Station Names: ID=7068, Data=Dockside Dr / Queens Quay E (Sugar Beach), Reference=Blantyre Park
9 Mismatch Station Names: ID=7208, Data=80 Clinton St (North of College) - SMART, Reference=80 Clinton St (North of College)
10 Mismatch Station Na

## Modification 2: Correct some station names with ' - SMART' at the end

In [82]:
station_id_list = [7324, 7342, 7208, 7039, 7094, 7123, 7326, 7301, 7141, 7353, 7403, 7449, 7445, 7450, 7408, 7447, 7415, 7438, 7414, 7441, 7459, 7451, 7412, 7423, 7482,
                   7435, 7480, 7410, 7434, 7437, 7425, 7429, 7413, 7476, 7492, 7431, 7466, 7448]
station_name_list = ['King St W / Charlotte St (West)', 'Morse St / Eastern Ave - SMART', '80 Clinton St (North of College) - SMART', 'Simcoe St / Michael Sweet Ave - SMART',
                     'Chester Station (Green P)', '424 Wellington St W - SMART', 'Davenport Rd / Lansdowne Ave.', 'Primrose Ave / Davenport Rd - SMART',
                     'Bridgeman Ave / Kendal Ave - SMART', 'Fulton Ave / Pape Ave - SMART', 'Lascelles Blvd / Eglinton Ave W - SMART', 'Cosburn Ave / Cedarvale Ave - SMART',
                     'Eastwood Rd / Coxwell Ave - SMART', 'Carlaw Ave / Danforth Ave - SMART', 'University Ave / Armoury St - SMART', 'Trent Ave / Danforth Ave - SMART',
                     'Keele St / Vine Ave - SMART', 'High Park Subway - SMART', 'Keele St / Annette St - SMART', 'Pacific Ave / Glenlake Ave - SMART', 'St Clair Ave W / Winona Ave - SMART',
                     'Western Battery Rd / Pirandello St - SMART', 'King St W / Crawford St - SMART', 'Merton St / Al Green Ln - SMART', 'Danforth Ave / Sibley Ave - SMART',
                     'Rushton Rd / St Clair Ave W - SMART', 'Neil McLellan Park - SMART', 'Howard St / Rose Ave - SMART', 'Via Italia / Mackay Ave - SMART',
                     'Concord Ave / Bloor St W - SMART', 'The Queensway at South Kingsway - SMART', 'Woodbine Subway Station - SMART', 'Dundas St W / Watkinson Ave - SMART',
                     'Symington Ave / Dupont St - SMART', '324 Cherry St - SMART', 'Gerrard St E / Leslie St - SMART', 'Glendonwynne Ave / Bloor St W - SMART', 'Goodwood Park Ct / Dentonia Park - SMART']

In [83]:
for i, id in enumerate(station_id_list):
  modified_bikeshare_stations.loc[modified_bikeshare_stations['station_id'] == id, ['name']] = station_name_list[i]

modified_bikeshare_stations.loc[modified_bikeshare_stations['station_id'] == 7448]

,station_id,name,lat,lon,capacity,address
404,7448,Goodwood Park Ct / Dentonia Park - SMART,43.694826,-79.295208,19.0,Goodwood Park Ct / Dentonia Park


# 5.0 Check 3: Station ID and Station Names + Mismatch Station Names with Same Station ID

In [84]:
modified_reference_station_dict = pd.Series(modified_bikeshare_stations['name'].values,index=modified_bikeshare_stations['station_id']).to_dict()

In [85]:
check_missing_ref_id(start_station_2019_dict, modified_reference_station_dict)

1 Station ID not in Reference: 7219, Station Name: Wolfpack - 7219
2 Station ID not in Reference: 7218, Station Name: Make Invisible - 7218
3 Station ID not in Reference: 7394, Station Name: Toronto Bike Shop
4 Station ID not in Reference: 7511, Station Name: PBSC-OPS


In [86]:
check_mismatch_names(start_station_2019_dict, modified_reference_station_dict)

1 Mismatch Station Names: ID=7381, Data=Roehampton Ave / Yonge St, Reference=Yonge St / Orchard View Blvd
2 Mismatch Station Names: ID=7051, Data=Wellesley St E / Yonge St (Green P), Reference=Mutual St / Shuter St
3 Mismatch Station Names: ID=7127, Data=Bay St / Davenport Rd, Reference=Bay St / Scollard St
4 Mismatch Station Names: ID=7372, Data=King St W / Portland St, Reference=Adelaide St W / Portland St
5 Mismatch Station Names: ID=7001, Data=Lower Jarvis St / The Esplanade, Reference=Wellesley Station Green P
6 Mismatch Station Names: ID=7068, Data=Dockside Dr / Queens Quay E (Sugar Beach), Reference=Blantyre Park
7 Mismatch Station Names: ID=7117, Data=Castle Frank Station, Reference=640 Bloor Street E
8 Mismatch Station Names: ID=7096, Data=Boston Ave / Queen St E, Reference=Lower Don River Trail and Taylor Creek Trail
9 Mismatch Station Names: ID=7258, Data=Queen St E / Larchmount Ave, Reference=Queen St E / Rushbrooke Ave
10 Mismatch Station Names: ID=7398, Data=York St / Har

|#|Station ID |lat|lon|
|---|---|---|---|
|1|7381|43.708156|-79.398610|
|2|7051|43.665115|-79.384562|
|3|7127|43.673263|-79.390655|
|4|7372|43.644561|-79.400121|
|5|7001|43.648368|-79.370940|
|6|7068|43.645298|-79.365583|
|7|7117|43.673836|-79.368891|
|8|7096|43.661558|-79.338723|
|9|7258|43.662500|-79.334297|
|10|7398|43.64122|-79.380711|

## Modification 3

In [90]:
station_id_list = [7381, 7051, 7127, 7372, 7001, 7068, 7117, 7096, 7258, 7398]
station_name_list = ['Roehampton Ave / Yonge St', 'Wellesley St E / Yonge St (Green P)', 'Bay St / Davenport Rd', 'King St W / Portland St', 
                     'Lower Jarvis St / The Esplanade', 'Dockside Dr / Queens Quay E (Sugar Beach)', 'Castle Frank Station', 'Boston Ave / Queen St E',
                     'Queen St E / Larchmount Ave', 'York St / Harbour St']
station_lat_list = [43.708156, 43.665115, 43.673263, 43.644561, 43.648368, 43.645298, 43.673836, 43.661558, 43.662500, 43.64122]
station_lon_list = [-79.398610, -79.384562, -79.390655, -79.400121, -79.370940, -79.365583, -79.368891, -79.338723, -79.334297, -79.380711]

In [92]:
print('Before: ')
modified_bikeshare_stations.loc[modified_bikeshare_stations['station_id'] == 7001]

Before: 


,station_id,name,lat,lon,capacity,address
1,7001,Wellesley Station Green P,43.664964,-79.38355,17.0,Yonge / Wellesley


In [93]:
for i, id in enumerate(station_id_list):
  modified_bikeshare_stations.loc[modified_bikeshare_stations['station_id'] == id, ['name', 'lat', 'lon', 'capacity', 'address']] = station_name_list[i], station_lat_list[i], station_lon_list[i], np.nan, ''

In [94]:
print('After: ')
modified_bikeshare_stations.loc[modified_bikeshare_stations['station_id'] == 7001]

After: 


,station_id,name,lat,lon,capacity,address
1,7001,Lower Jarvis St / The Esplanade,43.648368,-79.37094,NaN,


# 6.0 Check 4: Station ID and Station Names + Mismatch Station Names with Same Station ID

In [97]:
modified_reference_station_dict = pd.Series(modified_bikeshare_stations['name'].values,index=modified_bikeshare_stations['station_id']).to_dict()

In [100]:
check_missing_ref_id(start_station_2019_dict, modified_reference_station_dict)
check_mismatch_names(start_station_2019_dict, modified_reference_station_dict)

1 Station ID not in Reference: 7219, Station Name: Wolfpack - 7219
2 Station ID not in Reference: 7218, Station Name: Make Invisible - 7218
3 Station ID not in Reference: 7394, Station Name: Toronto Bike Shop
4 Station ID not in Reference: 7511, Station Name: PBSC-OPS


In [101]:
print(modified_bikeshare_stations.shape)
modified_bikeshare_stations.head()

(619, 6)


,station_id,name,lat,lon,capacity,address
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,35.0,Fort York Blvd / Capreol Ct
1,7001,Lower Jarvis St / The Esplanade,43.648368,-79.370940,NaN,
2,7002,St. George St / Bloor St W,43.667333,-79.399429,19.0,St. George St / Bloor St W
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761,15.0,Madison Ave / Bloor St W
4,7004,University Ave / Elm St,43.656518,-79.389099,11.0,University Ave / Elm St


In [102]:
print(data_2019_all_cleaned.shape)
data_2019_all_cleaned.head()

(2439047, 22)


,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Bike Id,Start Year,Start Month,Start Hour,Start Day of Week,Start Holiday,End Year,End Month,End Hour,End Day of Week,End Holiday,Start Day of Year,Start Week of Year
0,4581278,1547.0,7021,2019-01-01 00:08:00,Bay St / Albert St,7233,2019-01-01 00:33:00,King / Cowan Ave - SMART,Annual Member,1296,2019,1,0,1,1,2019,1,0,1,1,1,1
1,4581279,1112.0,7160,2019-01-01 00:10:00,King St W / Tecumseth St,7051,2019-01-01 00:29:00,Wellesley St E / Yonge St (Green P),Annual Member,2947,2019,1,0,1,1,2019,1,0,1,1,1,1
2,4581280,589.0,7055,2019-01-01 00:15:00,Jarvis St / Carlton St,7013,2019-01-01 00:25:00,Scott St / The Esplanade,Annual Member,2293,2019,1,0,1,1,2019,1,0,1,1,1,1
3,4581281,259.0,7012,2019-01-01 00:16:00,Elizabeth St / Edward St (Bus Terminal),7235,2019-01-01 00:20:00,Bay St / College St (West Side) - SMART,Annual Member,283,2019,1,0,1,1,2019,1,0,1,1,1,1
4,4581282,281.0,7041,2019-01-01 00:19:00,Edward St / Yonge St,7257,2019-01-01 00:24:00,Dundas St W / St. Patrick St,Annual Member,1799,2019,1,0,1,1,2019,1,0,1,1,1,1


# 7.0 Merging DataFrames

## Merge 1: Start Stations

In [111]:
merged_bike_data_2019 = pd.merge(left=data_2019_all_cleaned, right=modified_bikeshare_stations,
    how="outer", on=None, left_on=["Start Station Id", "Start Station Name"], right_on=["station_id", "name"],
    left_index=False, right_index=False, sort=False, suffixes=("_x", "_y"), copy=True,indicator=False,validate=None)

merged_bike_data_2019.drop(columns=["station_id", "name", "capacity", "address"], inplace=True)
merged_bike_data_2019.rename(columns={"lat" : "Start Lat", "lon" : "Start Lon"}, inplace=True)

print(merged_bike_data_2019.shape)
print(merged_bike_data_2019.columns)
merged_bike_data_2019.head()

(2439201, 24)
Index(['Trip Id', 'Trip Duration', 'Start Station Id', 'Start Time',
       'Start Station Name', 'End Station Id', 'End Time', 'End Station Name',
       'User Type', 'Bike Id', 'Start Year', 'Start Month', 'Start Hour',
       'Start Day of Week', 'Start Holiday', 'End Year', 'End Month',
       'End Hour', 'End Day of Week', 'End Holiday', 'Start Day of Year',
       'Start Week of Year', 'Start Lat', 'Start Lon'],
      dtype='object')


,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Bike Id,Start Year,Start Month,Start Hour,Start Day of Week,Start Holiday,End Year,End Month,End Hour,End Day of Week,End Holiday,Start Day of Year,Start Week of Year,Start Lat,Start Lon
0,4581278.0,1547.0,7021.0,2019-01-01 00:08:00,Bay St / Albert St,7233.0,2019-01-01 00:33:00,King / Cowan Ave - SMART,Annual Member,1296.0,2019.0,1.0,0.0,1.0,1.0,2019.0,1.0,0.0,1.0,1.0,1.0,1.0,43.653264,-79.382458
1,4581467.0,453.0,7021.0,2019-01-01 10:35:00,Bay St / Albert St,7253.0,2019-01-01 10:42:00,John St / Mercer St - SMART,Annual Member,1118.0,2019.0,1.0,10.0,1.0,1.0,2019.0,1.0,10.0,1.0,1.0,1.0,1.0,43.653264,-79.382458
2,4581748.0,632.0,7021.0,2019-01-01 14:13:00,Bay St / Albert St,7323.0,2019-01-01 14:24:00,457 King St. W. at Spadina,Annual Member,111.0,2019.0,1.0,14.0,1.0,1.0,2019.0,1.0,14.0,1.0,1.0,1.0,1.0,43.653264,-79.382458
3,4582209.0,234.0,7021.0,2019-01-01 19:05:00,Bay St / Albert St,7052.0,2019-01-01 19:09:00,Wellington St W / Bay St,Annual Member,942.0,2019.0,1.0,19.0,1.0,1.0,2019.0,1.0,19.0,1.0,1.0,1.0,1.0,43.653264,-79.382458
4,4583054.0,843.0,7021.0,2019-01-02 09:14:00,Bay St / Albert St,7036.0,2019-01-02 09:28:00,Trinity St /Front St E,Annual Member,1814.0,2019.0,1.0,9.0,2.0,0.0,2019.0,1.0,9.0,2.0,0.0,2.0,1.0,43.653264,-79.382458


In [113]:
count = 0
unresolved_station_ids = [7219, 7218, 7394, 7511]
for id in unresolved_station_ids:
  count += len(merged_bike_data_2019[merged_bike_data_2019['Start Station Id'] == id])
print("Number of instances with unresolved station IDs", count)

Number of instances with unresolved station IDs 74


In [114]:
merged_bike_data_2019.isnull().sum()

Trip Id               154
Trip Duration         154
Start Station Id      154
Start Time            154
Start Station Name    154
End Station Id        154
End Time              154
End Station Name      154
User Type             154
Bike Id               154
Start Year            154
Start Month           154
Start Hour            154
Start Day of Week     154
Start Holiday         154
End Year              154
End Month             154
End Hour              154
End Day of Week       154
End Holiday           154
Start Day of Year     154
Start Week of Year    154
Start Lat              74
Start Lon              74
dtype: int64

In [115]:
merged_bike_data_2019.dropna(inplace=True)
print(merged_bike_data_2019.shape)

(2438973, 24)


In [116]:
2439047 - 2438973

74

In [110]:
merged_bike_data_2019.isnull().sum()

Trip Id               0
Trip Duration         0
Start Station Id      0
Start Time            0
Start Station Name    0
End Station Id        0
End Time              0
End Station Name      0
User Type             0
Bike Id               0
Start Year            0
Start Month           0
Start Hour            0
Start Day of Week     0
Start Holiday         0
End Year              0
End Month             0
End Hour              0
End Day of Week       0
End Holiday           0
Start Day of Year     0
Start Week of Year    0
Start Lat             0
Start Lon             0
dtype: int64

## Merge 2: End Stations

In [117]:
merged_bike_data_2019 = pd.merge(left=merged_bike_data_2019, right=modified_bikeshare_stations,
    how="outer", on=None, left_on=["End Station Id", "End Station Name"], right_on=["station_id", "name"],
    left_index=False, right_index=False, sort=False, suffixes=("_x", "_y"), copy=True,indicator=False,validate=None)

merged_bike_data_2019.drop(columns=["station_id", "name", "capacity", "address"], inplace=True)
merged_bike_data_2019.rename(columns={"lat" : "End Lat", "lon" : "End Lon"}, inplace=True)

print(merged_bike_data_2019.shape)
print(merged_bike_data_2019.columns)
merged_bike_data_2019.head()

(2439128, 26)
Index(['Trip Id', 'Trip Duration', 'Start Station Id', 'Start Time',
       'Start Station Name', 'End Station Id', 'End Time', 'End Station Name',
       'User Type', 'Bike Id', 'Start Year', 'Start Month', 'Start Hour',
       'Start Day of Week', 'Start Holiday', 'End Year', 'End Month',
       'End Hour', 'End Day of Week', 'End Holiday', 'Start Day of Year',
       'Start Week of Year', 'Start Lat', 'Start Lon', 'End Lat', 'End Lon'],
      dtype='object')


,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Bike Id,Start Year,Start Month,Start Hour,Start Day of Week,Start Holiday,End Year,End Month,End Hour,End Day of Week,End Holiday,Start Day of Year,Start Week of Year,Start Lat,Start Lon,End Lat,End Lon
0,4581278.0,1547.0,7021.0,2019-01-01 00:08:00,Bay St / Albert St,7233.0,2019-01-01 00:33:00,King / Cowan Ave - SMART,Annual Member,1296.0,2019.0,1.0,0.0,1.0,1.0,2019.0,1.0,0.0,1.0,1.0,1.0,1.0,43.653264,-79.382458,43.637922,-79.431734
1,4586979.0,1243.0,7021.0,2019-01-03 17:13:00,Bay St / Albert St,7233.0,2019-01-03 17:34:00,King / Cowan Ave - SMART,Annual Member,3168.0,2019.0,1.0,17.0,3.0,0.0,2019.0,1.0,17.0,3.0,0.0,3.0,1.0,43.653264,-79.382458,43.637922,-79.431734
2,4591641.0,1112.0,7021.0,2019-01-04 21:42:00,Bay St / Albert St,7233.0,2019-01-04 22:00:00,King / Cowan Ave - SMART,Annual Member,3512.0,2019.0,1.0,21.0,4.0,0.0,2019.0,1.0,22.0,4.0,0.0,4.0,1.0,43.653264,-79.382458,43.637922,-79.431734
3,4594328.0,1156.0,7021.0,2019-01-05 23:21:00,Bay St / Albert St,7233.0,2019-01-05 23:40:00,King / Cowan Ave - SMART,Annual Member,212.0,2019.0,1.0,23.0,5.0,0.0,2019.0,1.0,23.0,5.0,0.0,5.0,1.0,43.653264,-79.382458,43.637922,-79.431734
4,4596322.0,1068.0,7021.0,2019-01-06 21:42:00,Bay St / Albert St,7233.0,2019-01-06 22:00:00,King / Cowan Ave - SMART,Annual Member,3279.0,2019.0,1.0,21.0,6.0,0.0,2019.0,1.0,22.0,6.0,0.0,6.0,1.0,43.653264,-79.382458,43.637922,-79.431734


In [118]:
count = 0
unresolved_station_ids = [7219, 7218, 7394, 7511]
for id in unresolved_station_ids:
  count += len(merged_bike_data_2019[merged_bike_data_2019['End Station Id'] == id])
print("Number of instances with unresolved station IDs", count)

Number of instances with unresolved station IDs 253


In [119]:
merged_bike_data_2019.isnull().sum()

Trip Id               155
Trip Duration         155
Start Station Id      155
Start Time            155
Start Station Name    155
End Station Id        155
End Time              155
End Station Name      155
User Type             155
Bike Id               155
Start Year            155
Start Month           155
Start Hour            155
Start Day of Week     155
Start Holiday         155
End Year              155
End Month             155
End Hour              155
End Day of Week       155
End Holiday           155
Start Day of Year     155
Start Week of Year    155
Start Lat             155
Start Lon             155
End Lat               253
End Lon               253
dtype: int64

In [120]:
merged_bike_data_2019.dropna(inplace=True)
print(merged_bike_data_2019.shape)

(2438720, 26)


In [121]:
2438973 - 2438720

253

In [122]:
merged_bike_data_2019.isnull().sum()

Trip Id               0
Trip Duration         0
Start Station Id      0
Start Time            0
Start Station Name    0
End Station Id        0
End Time              0
End Station Name      0
User Type             0
Bike Id               0
Start Year            0
Start Month           0
Start Hour            0
Start Day of Week     0
Start Holiday         0
End Year              0
End Month             0
End Hour              0
End Day of Week       0
End Holiday           0
Start Day of Year     0
Start Week of Year    0
Start Lat             0
Start Lon             0
End Lat               0
End Lon               0
dtype: int64

# 8.0 Final DataFrame

In [124]:
print(merged_bike_data_2019.shape)
print(merged_bike_data_2019.columns)
merged_bike_data_2019.head()

(2438720, 26)
Index(['Trip Id', 'Trip Duration', 'Start Station Id', 'Start Time',
       'Start Station Name', 'End Station Id', 'End Time', 'End Station Name',
       'User Type', 'Bike Id', 'Start Year', 'Start Month', 'Start Hour',
       'Start Day of Week', 'Start Holiday', 'End Year', 'End Month',
       'End Hour', 'End Day of Week', 'End Holiday', 'Start Day of Year',
       'Start Week of Year', 'Start Lat', 'Start Lon', 'End Lat', 'End Lon'],
      dtype='object')


,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Bike Id,Start Year,Start Month,Start Hour,Start Day of Week,Start Holiday,End Year,End Month,End Hour,End Day of Week,End Holiday,Start Day of Year,Start Week of Year,Start Lat,Start Lon,End Lat,End Lon
0,4581278.0,1547.0,7021.0,2019-01-01 00:08:00,Bay St / Albert St,7233.0,2019-01-01 00:33:00,King / Cowan Ave - SMART,Annual Member,1296.0,2019.0,1.0,0.0,1.0,1.0,2019.0,1.0,0.0,1.0,1.0,1.0,1.0,43.653264,-79.382458,43.637922,-79.431734
1,4586979.0,1243.0,7021.0,2019-01-03 17:13:00,Bay St / Albert St,7233.0,2019-01-03 17:34:00,King / Cowan Ave - SMART,Annual Member,3168.0,2019.0,1.0,17.0,3.0,0.0,2019.0,1.0,17.0,3.0,0.0,3.0,1.0,43.653264,-79.382458,43.637922,-79.431734
2,4591641.0,1112.0,7021.0,2019-01-04 21:42:00,Bay St / Albert St,7233.0,2019-01-04 22:00:00,King / Cowan Ave - SMART,Annual Member,3512.0,2019.0,1.0,21.0,4.0,0.0,2019.0,1.0,22.0,4.0,0.0,4.0,1.0,43.653264,-79.382458,43.637922,-79.431734
3,4594328.0,1156.0,7021.0,2019-01-05 23:21:00,Bay St / Albert St,7233.0,2019-01-05 23:40:00,King / Cowan Ave - SMART,Annual Member,212.0,2019.0,1.0,23.0,5.0,0.0,2019.0,1.0,23.0,5.0,0.0,5.0,1.0,43.653264,-79.382458,43.637922,-79.431734
4,4596322.0,1068.0,7021.0,2019-01-06 21:42:00,Bay St / Albert St,7233.0,2019-01-06 22:00:00,King / Cowan Ave - SMART,Annual Member,3279.0,2019.0,1.0,21.0,6.0,0.0,2019.0,1.0,22.0,6.0,0.0,6.0,1.0,43.653264,-79.382458,43.637922,-79.431734


Consideration:
* some docks might be in-service
    * account for uncertainty in breakdowns --> non-zero num_bikes_disabled